# NLP con Long-Short Term Memory (LSTM)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Ohtar10/icesi-nlp/blob/main/Sesion2/2-nlp-with-lstm.ipynb)

En este notebook implementaremos un clasificador de noticias en español utilizando la arquitectura de red LSTM. La idea es tener un punto de referencia para comparar cuando observemos la parte de transformers, por lo que utilizaremos el mismo dataset y tarea de ejemplo. Utilizarémos las utilidades de tokenización de huggingface transformers para ayudarnos con esta tarea.

#### Referencias
- Dataset: https://huggingface.co/datasets/MarcOrfilaCarreras/spanish-news
- [Long Short-Term Memory](https://www.researchgate.net/publication/13853244_Long_Short-Term_Memory#fullTextFileContent)

## 1. Chequeo de ambiente y Manejo de "Warnings"

En esta primer código verificamos si el ambiente de Trabajo es "Google Colab" e ignorar algun mensaje de "Warning" que suceda a lo resto del Notebook.

In [ ]:
import importlib.metadata    # Importación del modulo de metadata de la libreria importlib
import warnings               # Importación de la libreria warnings

# Uso de la libreria importlib.metadata para obtener los paquetes
# instalados en el Notebook
installed_packages = [dist.metadata['Name'].lower() for dist in importlib.metadata.Distribution.discover()]

# Chequeando que Google Colab este entre los paquetes instalados
# para confirmar si es el ambiente de trabajo usado.
IN_COLAB = 'google-colab' in installed_packages

# Ignorar algun Warning generado durante la ejecución del Notebook
warnings.filterwarnings('ignore')

### 2.

In [ ]:
!test '{IN_COLAB}' = 'True' && wget  !test '{IN_COLAB}' = 'True' && wget  /content/requirements.txt
!test '{IN_COLAB}' = 'True' && sudo apt-get update -y
!test '{IN_COLAB}' = 'True' && sudo apt-get install python3.10 python3.10-distutils python3.10-lib2to3 -y
!test '{IN_COLAB}' = 'True' && sudo update-alternatives --install /usr/local/bin/python python /usr/bin/python3.11 2
!test '{IN_COLAB}' = 'True' && sudo update-alternatives --install /usr/local/bin/python python /usr/bin/python3.10 1
!test '{IN_COLAB}' = 'True' && pip install lightning datasets

--2025-08-16 23:13:43--  http://!test/
Resolving !test (!test)... failed: Name or service not known.
wget: unable to resolve host address ‘!test’
--2025-08-16 23:13:43--  http://true/
Resolving true (true)... failed: Name or service not known.
wget: unable to resolve host address ‘true’
--2025-08-16 23:13:43--  http://=/
Resolving = (=)... failed: Name or service not known.
wget: unable to resolve host address ‘=’
--2025-08-16 23:13:43--  http://true/
Resolving true (true)... failed: Name or service not known.
wget: unable to resolve host address ‘true’
Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://ppa.launchpa

### Cargando el dataset
Este es un dataset pequeño de articulos de noticias en idioma español con sus respectivas categorías. El dataset está disponible en el HuggingFace Hub y puede ser fácilmente descargado con la librería.

In [ ]:
from datasets import load_dataset, Dataset
import warnings
import os
import pandas as pd

warnings.filterwarnings("ignore")
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

# Attempt to load the CSV using load_dataset with the correct delimiter
try:
    dataset = load_dataset('csv', data_files='/content/fake_news_dataset.csv', split='train', delimiter=';')
except Exception as e:
    print(f"Could not load with datasets: {e}")
    # Fallback to loading the CSV using pandas first if datasets.load_dataset fails
    df = pd.read_csv('/content/fake_news_dataset.csv', delimiter=';')
    # Convert the pandas DataFrame to a Hugging Face Dataset
    dataset = Dataset.from_pandas(df)

dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['title', 'text', 'date', 'source', 'author', 'category', 'label'],
    num_rows: 20000
})

In [ ]:
import pandas as pd

df = pd.read_csv('/content/fake_news_dataset.csv')
display(df.head())

,title;text;date;source;author;category;label
0,Foreign Democrat final.;more tax development b...
1,To offer down resource great point.;probably g...
2,Himself church myself carry.;them identify for...
3,You unit its should.;phone which item yard Rep...
4,Billion believe employee summer how.;wonder my...


In [ ]:
import pandas as pd

df = pd.read_csv('/content/fake_news_dataset.csv')
display(df.head())

,title;text;date;source;author;category;label
0,Foreign Democrat final.;more tax development b...
1,To offer down resource great point.;probably g...
2,Himself church myself carry.;them identify for...
3,You unit its should.;phone which item yard Rep...
4,Billion believe employee summer how.;wonder my...


Observemos uno de sus registros

In [ ]:
dataset[1]

{'title': 'To offer down resource great point.',
 'text': 'probably guess western behind likely next investment consumer range wrong exactly once attack shoulder movie partner daughter on executive tonight factor push development pass question field firm accept I represent answer computer win fast small character total myself air must difficult green fast writer adult though individual learn interview our available drug against group produce before large wish find even media nature then last computer project story special stand lead build during ball contain road since history customer garden figure kind throw tell discuss remain view morning put mouth while serve great certain free two structure skin yard position suffer fast someone ok mind must something outside position write theory ok letter for debate seat top fall authority bit deep there get man view loss bring friend free certain economic final occur summer similar best discover area real area still scientist social everybody 

Para los efectos de esta tarea, nos servirán el texto y la categoría naturalmente.

A manera general, observemos que tan largos o cortos tienden a ser los textos.

In [ ]:
text_lengths = [len(row['text']) for row in dataset]
print(f"Texto más corto: {min(text_lengths)}")
print(f"Texto más largo: {max(text_lengths)}")
print(f"Longitud promedio: {sum(text_lengths) / len(text_lengths)}")

Texto más corto: 1223
Texto más largo: 2077
Longitud promedio: 1635.1159


Estos valores son la cantidad de *caractéres* que tiene las secuencias. Una decisión ingenua pero útil en este momento podría ser ajustar la longitud de las secuencias que vamos a usar para el entrenamiento a unos 2000 tokens. Esto podría ser suficiente para capturar una porción significativa de los textos.

## Definiendo el Tokenizer

Ahora, vamos a definir el tokenizer para nuestra tarea. Para mantener las cosas simples, vamos a mantener un conteo de palabras y vamos a hacer un corte hasta los primeros 50mil tokens.

In [ ]:
import re
from collections import Counter

def simple_tokenizer(text):
    text = text.lower()
    text = re.sub(r"[^a-záéíóúüñ]+", " ", text)
    return text.strip().split()

# Construimos el vocabulario a partir de conjunto de datos.
token_counts = Counter()
for text in dataset["text"]:
    token_counts.update(simple_tokenizer(text))

# 50k-2 porque necesitamos reservar espacio para los dos tokens especiales
top_n_tokens = list(token_counts.keys())[:50000-2]
vocab = {"[PAD]": 0, "[UNK]": 1}
for token in top_n_tokens:
    vocab[token] = len(vocab)

def tokenize_text(text, max_length=50):
    tokens = simple_tokenizer(text)
    ids = [vocab.get(tok, vocab["[UNK]"]) for tok in tokens[:max_length]]
    ids += [vocab["[PAD]"]] * (max_length - len(ids))
    return ids

Exploremos ahora el tokenizador obtenido.

In [ ]:
print(f"Vocabulario: {len(vocab)} tokens")
print("Primeros 15 tokens:")
print(f"{top_n_tokens[:15]}")
print("15 tokens de en medio:")
print(f"{top_n_tokens[1000:1015]}")
print("Últimos 15 tokens:")
print(f"{top_n_tokens[-15:]}")

Vocabulario: 973 tokens
Primeros 15 tokens:
['more', 'tax', 'development', 'both', 'store', 'agreement', 'lawyer', 'hear', 'outside', 'continue', 'reach', 'difference', 'yeah', 'figure', 'your']
15 tokens de en medio:
[]
Últimos 15 tokens:
['grow', 'dark', 'whole', 'any', 'off', 'it', 'well', 'oil', 'scene', 'action', 'prepare', 'live', 'who', 'report', 'event']


Esta forma de exploración es para darnos una idea de las palabras más utilizadas en el corpus y nos dará un indicio de si la tokenización es adecuada o no. Vemos que tenemos algunos stop words, como artículos (el, la) y conectores (del, que). Para una tarea de clasificación de texto podríamos prescindir de estos pero para facilitar las cosas y ya que los demás tokens lucen bien, podemos preservarlos.

Ahora veamos como convierte el tokenizador una oración muy sencilla:

In [ ]:
tokenized = tokenize_text("probably guess", max_length=8)
tokenized

[191, 192, 0, 0, 0, 0, 0, 0]

Lo que obtenemos de vuelta son los ids de cada token según el vocabulario. Ahora algo importante que notamos aquí es el *padding*, durante el entrenamiento, queremos que las secuencias sean de tamaño fijo, para asi operar comodamente con matrices. Pero ya vimos que no todos los textos tienen la misma longitud. Entonces que hacer? para los que son más largos que una longitud dada simplemente cortamos, pero para los que son más cortos, debemos *rellenar* lo faltante con un *token especial de relleno o padding*. Y es justo lo que definimos allí, cuando la cadena es inferior a 8 **tokens**, entonces debemos hacer padding hasta que se cumplan los 8.

Si queremos saber a que token exactamente hacen referencia estos ids, simplemente revisamos el vocabulario que hemos construido:

In [ ]:
id_2_token = {v: k for k, v in vocab.items()}
[id_2_token[token] for token in tokenized]

['probably', 'guess', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']

Claramente vemos los 3 tokens como cadenas independientes (el padding se considera un token independiente).

### Definiendo el dataset de pytorch
Ahora podemos proceder a definir el dataset. Esto debería ser muy sencillo dado que nuestro dataset es pequeño y ya tenemos el tokenizador listo.

In [ ]:
import torch
import numpy as np
from typing import Tuple, Dict
from torch.utils.data import Dataset

class SpanishNewsDataset(Dataset):

    def __init__(self, tokenizer, dataset, seq_length: int = 512):
        self.tokenizer = tokenizer
        self.dataset = dataset
        self.seq_length = seq_length
        # Definimos estos dos mapas para facilitarnos la tarea
        # de traducir de nombres de categoría a ids de categoría.
        self.id_2_class_map = dict(enumerate(np.unique(dataset[:]['category'])))
        self.class_2_id_map = {v: k for k, v in self.id_2_class_map.items()}
        self.num_classes = len(self.id_2_class_map)

    def __getitem__(self, index) -> Dict[str, torch.Tensor]:
        text, y = self.dataset[index]['text'], self.dataset[index]['category']
        y = self.class_2_id_map[y]
        data = {'input_ids': torch.tensor(self.tokenizer(text, max_length=self.seq_length))}
        data['y'] = torch.tensor(y)
        return data


    def __len__(self):
        return len(self.dataset)

Ahora instanciaremos el dataset entero. Para este experimento, definiremos un tamaño máximo de secuencia de 2048 **tokens**. Que según nuestra intuición arriba, debería ser suficiente para la tarea.

In [ ]:
max_len = 512

# Filter out rows where 'category' is None
filtered_dataset = dataset.filter(lambda example: example['category'] is not None)

spanish_news_dataset = SpanishNewsDataset(tokenize_text, filtered_dataset, seq_length=max_len)
assert len(spanish_news_dataset) == len(filtered_dataset)

Filter:   0%|          | 0/20000 [00:00<?, ? examples/s]

Y luego, procedemos a hacer el train-val-test split y crear los dataloaders.

In [ ]:
from torch.utils.data import random_split
from torch.utils.data import DataLoader

batch_size = 4 if not IN_COLAB else 12
train_dataset, val_dataset, test_dataset = random_split(spanish_news_dataset, lengths=[0.8, 0.1, 0.1])
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=2)

## Definición del modelo LSTM

Ahora vamos a configurar un módulo pytorch simple para este problema. Vamos ha utilizar los embeddings, que vendrían siendo los vectores de palabra. Pytorch nos ofrece una capa con la que directamente podemos entrenarlos a partir de los token ids obtenidos. El resto consistirá en invocar una capa LSTM seguida de una capa densa para la clasificación.

Recordemos que las redes recurrentes como las LSTM por diseño enlazan todas las dimensiones del vector de entrada, formando así la secuencia, la estructura natural que necesitamos representar.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LSTMBlock(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes, num_layers=2, dropout=0.2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)

    def forward(self, x):
        embedded = self.embedding(x)
        output, (hidden, _) = self.lstm(embedded)
        return hidden[-1]


### Definición del clasificador

Finalmente, definimos el modelo en si. Este modelo constará de 3 capas:

- La tokenización, tal como la definimos anteriormente.
- El bloque LSTM, que acabamos de decinir.
- Una capa densa adicional que servirá como clasificador de aquello que nos entregue la capa del transformer.

Como este es un LightningModule, aquí definiremos el resto de funciones utilitarias para el entrenamiento de la tarea.

In [ ]:
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torchmetrics import Accuracy
import torch

class SpanishNewsClassifierWithLSTM(LightningModule):

    def __init__(self, vocab_size: int, num_classes: int, emb_dim: int, hidden_dim: int = 128):
        super(SpanishNewsClassifierWithLSTM, self).__init__()
        self.num_classes = num_classes
        self.lstm = LSTMBlock(vocab_size, emb_dim, hidden_dim, num_classes)
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(hidden_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes),
            nn.LogSoftmax(dim=1)
        )

        self.train_acc = Accuracy(task='multiclass', num_classes=num_classes)
        self.val_acc = Accuracy(task='multiclass', num_classes=num_classes)
        self.test_acc = Accuracy(task='multiclass', num_classes=num_classes)

    def forward(self, x):
        embeddings = self.lstm(x)
        return self.classifier(embeddings)


    def training_step(self, batch, batch_idx):
        x, y = batch['input_ids'], batch['y']
        # print(f"\nbatch-idx: {batch_idx}")
        # print(f"shape of x: {x.shape}")
        # print(torch.max(x, dim=0))
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y.long()) # Cast y to LongTensor
        self.train_acc(y_hat, y)
        self.log('train-loss', loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log('train-acc', self.train_acc, prog_bar=True, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, batch):
        x, y = batch['input_ids'], batch['y']
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y.long()) # Cast y to LongTensor
        self.val_acc(y_hat, y)
        self.log('val-loss', loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log('val-acc', self.val_acc, prog_bar=True, on_step=False, on_epoch=True)
        return loss

    def test_step(self, batch):
        x, y = batch['input_ids'], batch['y']
        y_hat = self(x)
        self.test_acc(y_hat, y)
        self.log('test-acc', self.test_acc, prog_bar=True, on_step=False, on_epoch=True)


    def predict_step(self, batch):
        x = batch['input_ids']
        return self(x)


    def configure_optimizers(self):
        optimizer =  torch.optim.AdamW(self.parameters(), lr=1e-3, weight_decay=1e-5)
        return optimizer


model = SpanishNewsClassifierWithLSTM(vocab_size=len(vocab) + 1, num_classes=spanish_news_dataset.num_classes, emb_dim=256)

tb_logger = TensorBoardLogger('tb_logs', name='LSTMClassifier')
callbacks=[EarlyStopping(monitor='train-loss', patience=3, mode='min')]
trainer = Trainer(max_epochs=10, accelerator='gpu', devices=1, logger=tb_logger, callbacks=callbacks)

trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type               | Params | Mode 
----------------------------------------------------------
0 | lstm       | LSTMBlock          | 579 K  | train
1 | classifier | Sequential         | 199 K  | train
2 | train_acc  | MulticlassAccuracy | 0      | train
3 | val_acc    | MulticlassAccuracy | 0      | train
4 | test_acc   | MulticlassAccuracy | 0      | train

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Observemos el proceso de entrenamiento

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir tb_logs/

<IPython.core.display.Javascript object>

Y como es de esperarse, realizaremos la validación contra el conjunto de prueba.

In [ ]:
model.eval()
trainer.test(model, test_loader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test-acc          │    0.14300000667572021    │
└───────────────────────────┴───────────────────────────┘

[{'test-acc': 0.14300000667572021}]

### Haciendo predicciones

Finalmente, vamos a hacer uso del modelo y ver que tan bueno es para la clasificación de noticias.

In [ ]:
predictions = trainer.predict(model, test_loader)
predictions = torch.cat(predictions, dim=0)
predictions = torch.argmax(predictions, dim=-1)
predictions = [spanish_news_dataset.id_2_class_map[pred] for pred in predictions.numpy()]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [ ]:
import pandas as pd

test_indices = test_dataset.indices
df = pd.DataFrame(data={
    "titulo": dataset[test_indices]['title'],
    "tokens": [tokenize_text(v) for v in dataset[test_indices]['text']],
    "fecha": dataset[test_indices]['date'],
    "source": dataset[test_indices]['source'],
    "category": dataset[test_indices]['category'],
    "label": dataset[test_indices]['label'],
    'predicción': predictions
}, index=test_indices)

df['tokens_string'] = df.tokens.apply(lambda t: ' '.join([id_2_token[i] for i in t]))
df = df[["titulo", "tokens", "tokens_string", "category", "label", "predicción"]]
df.style.set_table_styles(
    [
        {'selector': 'td', 'props': [('word-wrap', 'break-word')]}
    ]
)
display(df.head(15))

,titulo,tokens,tokens_string,category,label,predicción
11101,Than newspaper face audience can may second.,"[635, 110, 144, 904, 3, 343, 837, 555, 859, 57...",with we us without tax relate able these ever ...,Sports,fake,Health
4120,Building we month determine table form.,"[865, 19, 943, 339, 820, 640, 510, 15, 391, 87...",popular identify west worry feeling around tas...,Technology,fake,Health
19407,Ago customer system before piece as.,"[495, 751, 293, 266, 401, 76, 375, 125, 691, 9...",either strategy scientist structure himself pa...,Business,fake,Health
6813,Movement reduce different investment factor ou...,"[386, 516, 965, 344, 477, 198, 633, 810, 449, ...",despite couple oil amount kid wrong administra...,Health,fake,Health
11504,Affect movement commercial degree.,"[473, 631, 237, 116, 141, 956, 367, 804, 602, ...",trial between wish hotel especially create pre...,Sports,real,Health
19574,Third next value skill.,"[884, 387, 463, 97, 880, 403, 698, 421, 644, 6...",agent less choice talk out follow keep toward ...,Sports,fake,Health
1487,When ask life four north somebody smile.,"[573, 929, 142, 603, 503, 818, 42, 218, 845, 1...",let democrat north wonder cover plan ahead mys...,Sports,fake,Health
18889,Century listen window rule cold.,"[201, 396, 692, 896, 515, 736, 672, 341, 718, ...",daughter much have the speech every through ma...,Health,fake,Health
8939,Agreement include seem.,"[643, 731, 738, 273, 205, 784, 963, 960, 218, ...",city usually environmental mind factor center ...,Entertainment,real,Health
19088,Serve cultural record effect discover sing cos...,"[904, 607, 117, 972, 511, 5, 953, 191, 861, 61...",without various cut event finally both light p...,Technology,real,Health


In [ ]:
errors = df[df['category'] != df['predicción']]
errors.head(15)

,titulo,tokens,tokens_string,category,label,predicción
11101,Than newspaper face audience can may second.,"[635, 110, 144, 904, 3, 343, 837, 555, 859, 57...",with we us without tax relate able these ever ...,Sports,fake,Health
4120,Building we month determine table form.,"[865, 19, 943, 339, 820, 640, 510, 15, 391, 87...",popular identify west worry feeling around tas...,Technology,fake,Health
19407,Ago customer system before piece as.,"[495, 751, 293, 266, 401, 76, 375, 125, 691, 9...",either strategy scientist structure himself pa...,Business,fake,Health
11504,Affect movement commercial degree.,"[473, 631, 237, 116, 141, 956, 367, 804, 602, ...",trial between wish hotel especially create pre...,Sports,real,Health
19574,Third next value skill.,"[884, 387, 463, 97, 880, 403, 698, 421, 644, 6...",agent less choice talk out follow keep toward ...,Sports,fake,Health
1487,When ask life four north somebody smile.,"[573, 929, 142, 603, 503, 818, 42, 218, 845, 1...",let democrat north wonder cover plan ahead mys...,Sports,fake,Health
8939,Agreement include seem.,"[643, 731, 738, 273, 205, 784, 963, 960, 218, ...",city usually environmental mind factor center ...,Entertainment,real,Health
19088,Serve cultural record effect discover sing cos...,"[904, 607, 117, 972, 511, 5, 953, 191, 861, 61...",without various cut event finally both light p...,Technology,real,Health
13069,Knowledge by sing government within agreement.,"[329, 14, 578, 75, 195, 240, 841, 741, 269, 18...",unit yeah old of next media use color position...,Business,real,Health
9699,Tax current skill film rate six build.,"[282, 655, 446, 282, 248, 124, 662, 615, 150, ...",loss four never loss during information court ...,Sports,real,Health


## Conclusiones

- En este caso tenemos una tarea de clasificación de texto de dos clases.
- Estamos usando un bloque LSTM como featurizer, es decir lo usamos para extraer features de las secuencias de entrada con las cuales harémos predicciones luego.
- Nótese que de las capas LSTM, solo nos interesa la última, ya que esta recupera todas las operaciones enalazadas anteriores.
- Observamos que el modelo toma su tiempo en entrenar, esto es natural debido al diseño de las LSTM, donde por cada paso de tiempo se debe computar un gradiente, por lo que el computo es mucho mayor.